In [1]:
import numpy as np

In [28]:
def fairshare1(v0,alloc):
    v = v0
    velem = []                                 ##############################
    for i in v0:                               #
        if i > 0:                              #
            velem.append(1)                    # velem = v > 0;
        else:                                  # Nelem = sum(velem);
            velem.append(0)                    #
    Nelem = sum(velem)                         #
    velem = np.array(velem)                    ##############################
        
    if Nelem != 0:
        chk = 0
        while chk == 0:
            f = alloc/Nelem                    # Fairshare allocation
            v = v - f*velem
            if min(v)<0:                       # some elements were overserved
                
                elemtemp = []                  ##############################
                vv = []                        # 
                for i in v:                    #
                    if i<0:                    #
                        elemtemp.append(1)     #
                        vv.append(i)           # alloc = -1*sum(v(elemtemp)); 
                    else:                      #
                        elemtemp.append(0)     #
                elemtemp = np.array(elemtemp)  #
                vv = np.array(vv)              #
                alloc = -1*sum(vv)             ##############################
                
                for i in range(0,len(v)):      ##############################
                    if v[i]<0:                 # v(elemtemp) = 0;
                        v[i] = 0               ##############################
                
                elemtemp = []                  ##############################
                for i in v:                    #
                    if i>0:                    #
                        elemtemp.append(1)     # velem = v > 0;
                    else:                      #
                        elemtemp.append(0)     #
                elemtemp = np.array(elemtemp)  #############################
                
                Nelem = sum(velem)             
                if Nelem == 0:
                    chk = 1
            else:                              # no elements were overserved
                chk = 1
    vfs = v0 - v
    return vfs
                
                

In [9]:
def RotorLB(N, Nmatch, NS, m, D, Ncycles):
    # N(int), Nmatch(int), NS(int), m(np.array (2D)), D(np.array (2D))
    queues = [[0]*3]*N
    for i in range(0,N):
        queues[i][0] = D[i]                  # Direct queues
        queues[i][1] = np.zeros(shape=(N,N)) # Indirect queues
        queues[i][2] = np.zeros(shape=(N,N)) # Temp queues
    queues = np.array(queues)
    
    # Simulation
    tol = 1e-8                               # Tolreance to round extra link capacity
    BWserved = np.zeros(1,Ncycles)
    
    for a in range(0,Ncycles):               # Iterate through cycles of matchings
        delivered = np.zeros(shape =(N,N))   # Initialize delivered banwidth for this cycle
        for b in range(0,Nmatch):            # Iterate through matchings
                                             # Initialize temp queues - these will be used to update indirect queues
            for x in range(0,N):
                queues[x][2] = np.zeros(shape = (N,N))
            # initialize link capacities
            mm = m[b]
            con = np.argwhere(mm[0]>0)
            Ncon = len(con)                  # Get number of connections
            linkcap = 1/NS                   # Fraction of ToR BW in each RotorNet link
            R =  linkcap*np.ones(shape = (N,Ncon))
            
            # -------------------  Phase 1 ------------------- #
            # First, serve any existing nonlocal (two hop) traffic
            
            for c in range(0,N):             # Sweep local nodes
                mm = m[b]
                con = np.argwhere(mm[c]>0)   # Get ids of connected nodes
                
                for d in range(0,Ncon):      # Sweep recivers
                    qq = queues[c][2]
                    vserved = fairshare1(qq[:,con[d]], R[c][d])
                    
                    # Update delivered bandwidth
                    delivered[:,con[d]] = delivered[:,con[d]] - vserved
                    # Update nonlocal queues
                    queues[c][2][:,con[d]] = queues[c][2][:,con[d]] - vserved
                    # Remaining capacity on the link
                    R[c][d] = R[c][d] - sum(vserved)
            
            # Next, send local data directly if possible
            for c in range(0,N):             # Sweep local nodes
                mm = m[b]
                con = np.argwhere(mm[c]>0)   # Get ids of connected nodes
                for d in range(0,Ncon):
                    if R[c][d] > 0:
                        if R[c][d] > queues[c][1][con[d]][0]:
                            delivered[c][con[d]] = delivered[c][con[d]] + queues[c][1][con[d]][0]